In [189]:
import os
import bs4
from bs4 import BeautifulSoup as soup
import pandas as pd
import spacy
import re
import en_core_web_sm
import keras
import tensorflow
from pathlib import Path
from pathlib import PureWindowsPath
import helper_parsing_functions
from helper_parsing_functions import *
import csv
from dateparser.search import search_dates
import arrow
import datetime

nlp = en_core_web_sm.load()

def extract_name_pair(name, input_text):
    out_list = []
    try:
        if not (input_text==[] or name==[]):
            #         s_out = extract_chunks(input_text)  
            sent =  " ".join(str(x) for x in input_text)
            #     doc = nlp(input_text)
            doc = nlp(sent)        
            #         doc = nlp(input_text)
            spans = list(doc.ents)  #+ list(doc.noun_chunks)

            for item in spans:
                try:
                    item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
                    if (re.search(item,name,flags=re.IGNORECASE)):
                        out_list.append(item)

                except:
                    pass
                return out_list
        else:
            return out_list
    except:
        return None

def extract_time(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.find_all('p'):
        temp_string=element.text.strip()
        
        try:
#             December 2, 2014 11:15 AM ET
            
            temp_string = re.match(string=temp_string,pattern=\
                                   '^[^a-zA-Z]*([a-zA-Z]+\s+[0-9]+,\s+[0-9]{4})[^ 0-9]?(\s+[0-9]{1,2}:[0-9]{2}\s+[APap]\.?[mM]\.?).*')
            string1 = temp_string.group(1)
            string2 = temp_string.group(2)
            #         name_search.group(1).strip()
            string3 = string1 + string2
            string3 = re.sub(string=string3,pattern='\.',repl='')
            string3 = re.sub(string=string3,pattern='\s+',repl=' ')
#             print(string1,'##',string2,'##',string3)
            date_out = arrow.get(string3,'MMMM D, YYYY H:mm A')
        except:
            date_out=None        
            
        if date_out is not None:
            return date_out.isoformat()
    temp = input_soup.find(id='article_content')
    
    for element in temp.find_all('a'):
        temp_string=element.text.strip()
        temp_string = re.sub(string=temp_string,pattern='\s+',repl=' ')
        try:
            while  (element):
                try:
                    date_out = arrow.get(temp_string,'MMMM D, YYYY H:mm A')
                except:
                    date_out=None       
                if date_out is not None:
    #                 return temp_string
                    return date_out.isoformat()


                element = element.next_sibling
                temp_string = element
                temp_string = re.sub(string=temp_string,pattern='\s+',repl=' ')  
    #             print(temp_string.find('div'))
    #             print(temp_string.find('strong'))
        except:
            pass
    return None

In [190]:
collection = "html"

# n_test=50
file_list = os.listdir(collection)
# for i in range(len(file_list)):
# for i in range(n_test):
#     print(file_list[i])
print(len(file_list))

3


In [174]:
collection = "./html"
# data_folder = Path(collection)
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
# input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')

output_folder = Path('./output/')
input_folder = Path(collection)


n_test=1
file_list = os.listdir(collection)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
print(len(file_list))

FILE	ANALYST_Flag	FIRM	Quarter	Year
3


In [175]:
out_list = []
for i in range(len(file_list)):
    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
        out_list.append(file_list[i])
        print(file_list[i])


5717.html
1.html


In [176]:
out_file = pd.DataFrame(out_list,columns=['filename'])
out_file.index.names=['index']
out_file.to_csv('all_html.csv',encoding='utf-8')

In [ ]:
out_file.iloc[0]['filename']
len(out_file)

In [191]:
out_file = pd.read_csv('/Users/lky/all_html.csv',names=['filename'])

In [192]:
out_file.head()


,filename
0,1.html
1,10.html
2,100.html
3,1000.html
4,10000.html


In [185]:
output_folder = Path('./sample_out')
output_folder_analyst = Path('./analyst_out')
output_folder_exec = Path('./exec_out')
output_folder_raw = Path('./')
input_folder = Path('/Volumes/S/convoparse/html/')
# input_folder = Path(collection)
# print(input_folder)

In [179]:
display(output_folder)
with open(str( output_folder/'test_extract.csv'), "w+") as csvfile:
    print('f')
    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    logwriter.writerow(['1','1','fail'])

PosixPath('sample_out')

f


In [194]:
ii = 0
test =[out_file.iloc[0].filename]
test
test = pd.DataFrame(test,columns=['filename'])
test
type(test)

pandas.core.frame.DataFrame

In [ ]:
output_folder = Path('./sample_out')

with open(str(output_folder / 'test_extract.csv'), "w+") as csvfile:

    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    i_start = 0
    i_end = len(out_file)
 
    for ii in range(i_start,i_end):
        try:
            #batch
            #basically 1 file at a time now
            row_start = ii*1
            row_end = (ii*1)+1
            

            output_filename = 'out_' + str(ii) + '.csv'
            output_filename_exec = 'out_exec_' + str(ii) + '.csv'
            output_filename_analyst = 'out_analyst_' + str(ii) + '.csv'

#             for i in range(row_start,row_end):
#                 out_list.append(file_list[i])
#             print('here')
            out_list =[]

            out_list.append(out_file.iloc[ii]['filename'])
            filename = input_folder /  out_file.iloc[ii]['filename']
            contents = extract_html(str(filename))
            
            out_pd = pd.DataFrame(out_list,columns=['filename'])
#             out_pd = out_file
            out_pd['text'] = out_pd.apply(lambda x: extract_html(str(input_folder / x['filename'])), axis=1)
            out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
            out_pd['call_date'] = out_pd.apply(lambda x: extract_time(x['text']), axis=1)
            out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)
            out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
            out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

            out_pd \
    .to_csv(output_folder_raw / output_filename,sep=',', encoding = 'utf-8',header=False)
            
            s = out_pd.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('executives')
            out_executives = out_pd.join(s)                      
            

            
            
            out_executives['exec_name'] = out_executives.apply(lambda x: extract_name(x['executives']), axis=1)
            out_executives['exec_title'] = out_executives.apply(lambda x: extract_title(x['executives']), axis=1)
#             out_pd = out_pd.drop(['text'],axis=1)

            out_executives['executive_content'] = out_executives.apply(lambda x: \
                                                     extract_content(contents,x['exec_name']), axis=1)
            out_executives['executive_qanda'] = out_executives.apply(lambda x: \
                                                     extract_qanda(contents,x['exec_name']), axis=1)

            out_executives \
            .drop(['text'],axis=1) \
            .to_csv(output_folder_raw / output_filename_exec,sep=',', encoding = 'utf-8',header=False)         
                
            out_executives=out_executives.reset_index(level=0, drop=True)
            
            s = out_executives.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('mention')
            out_executives = out_executives.join(s)    
            
            out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention']), axis=1)
            
            out_executives['mention_title'] = out_executives.apply(lambda x: extract_title(x['mention_name']), axis=1)
            
            out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
            
            out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
            

        
            
            s = out_pd.analysts.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('analyst')
            out_analysts = out_pd.join(s)                 
            
            
            out_analysts['analyst_name'] = out_analysts.apply(lambda x: extract_name(x['analyst']), axis=1)
            out_analysts['analyst_firm'] = out_analysts.apply(lambda x: extract_title(x['analyst']), axis=1)

            out_analysts['analyst_content'] = out_analysts.apply(lambda x: \
                                                     extract_content(contents,x['analyst_name']), axis=1)
            
            out_analysts['analyst_qanda'] = out_analysts.apply(lambda x: \
                                                     extract_qanda(contents,x['analyst_name']), axis=1)

            out_analysts \
            .drop(['text'],axis=1) \
            .to_csv(output_folder_raw / output_filename_analyst,sep=',', encoding = 'utf-8',header=False)   

            
            out_analysts=out_analysts.reset_index(level=0, drop=True)

            s = out_analysts.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('mention')
            out_analysts = out_analysts.join(s)                 
            out_analysts['mention_name'] = out_analysts.apply(lambda x: extract_name(x['mention']), axis=1)
            out_analysts['mention_title'] = out_analysts.apply(lambda x: extract_title(x['mention']), axis=1)
            
            
#             out_analysts \
#     .to_csv(output_folder / output_filename_analyst,sep=',', encoding = 'utf-8',header=True)
    
            
            
            out_analysts['exec_mention_qna'] = out_analysts.apply(lambda x: extract_name_pair(x['mention_name'],x['analyst_qanda']), axis=1)
            out_analysts['exec_mention_transcript'] = out_analysts.apply(lambda x: extract_name_pair(x['mention_name'],x['analyst_content']), axis=1)
            
            
#             out_executives['exec_qna_count'] = out_executives.apply(lambda x: \
#                                                      basic_wc(x['executive_qanda']), axis=1)

#             out_executives['exec_content_count'] = out_executives.apply(lambda x: \
#                                                      basic_wc(x['executive_content']), axis=1)
#             out_executives['exec_total_count'] = out_executives.exec_qna_count + out_executives.exec_content_count

#             out_pd=out_pd.reset_index(level=0, drop=True)
#             # out_analyst
#             s = out_pd.execs.apply(pd.Series)\
#                           .stack()\
#                           .reset_index(level=1, drop=True)\
#                           .to_frame('mention_name')
#             out_pd = out_pd.join(s)                      
                        
#             out_executives = out_executives.join(s)          
#             out_executives['mention_titile'] = out_executives.apply(lambda x: extract_title(x['mention_name']), axis=1)
#             out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention_name']), axis=1)
#             out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
#             out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
#             out_executives['mention_name_count_qna'] = out_executives.apply(lambda x: \
#                                           len(x['exec_mention_qna']), axis=1)
#             out_executives['mention_name_count_transcript'] = out_executives.apply(lambda x: \
#                                           len(x['exec_mention_transcript']), axis=1)
#             out_executives['mention_name_count_total']= out_executives.mention_name_count_qna+out_executives.mention_name_count_transcript
# #             print('here##')
#             # out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
#             out_executives.drop(['text','executive','execs','exec_mention_qna',\
#                                  'exec_mention_transcript'],axis=1)\
#             .to_csv(output_folder / filename,sep=',', encoding = 'utf-8',header=False)
           
#         #             .to_csv(filename,sep=',', encoding = 'utf-8',header=False)
#             # out_executives.select({[}'file','quarter_yer'])    
# #             print('success: ', ii, ' ###')
#             logwriter.writerow([ii,out_executives.file.head(1).item(),'success'])
           
            
#             out_executives \
#     .to_csv(output_folder / output_filename_exec,sep=',', encoding = 'utf-8',header=True)    
    
            out_executives \
    .drop(['text'],axis=1) \
    .to_csv(output_folder_exec / output_filename_exec,sep=',', encoding = 'utf-8',header=False)    
            
#             out_analysts \
#     .to_csv(output_folder / output_filename_analyst,sep=',', encoding = 'utf-8',header=True)
    
            out_analysts \
    .drop(['text'],axis=1) \
    .to_csv(output_folder_analyst / output_filename_analyst,sep=',', encoding = 'utf-8',header=False)  
            logwriter.writerow([ii,out_file.iloc[ii].filename,'success'])
        except:
            print('exception: ', ii, ' ###')
            logwriter.writerow([ii,out_file.iloc[ii].filename,'fail'])
            
            pass

exception:  2  ###
exception:  20  ###
exception:  29  ###
exception:  46  ###
exception:  51  ###
exception:  54  ###
exception:  64  ###
exception:  65  ###
exception:  71  ###
exception:  88  ###
exception:  93  ###
exception:  97  ###
exception:  109  ###
exception:  115  ###
exception:  118  ###
exception:  120  ###
exception:  123  ###
exception:  124  ###
exception:  129  ###
exception:  135  ###
exception:  153  ###
exception:  159  ###
exception:  167  ###
exception:  168  ###
exception:  180  ###
exception:  194  ###
exception:  200  ###
exception:  203  ###
exception:  221  ###
exception:  223  ###
exception:  228  ###
exception:  238  ###
exception:  239  ###
exception:  241  ###
exception:  246  ###
exception:  248  ###
exception:  283  ###
exception:  303  ###
exception:  317  ###
exception:  321  ###
exception:  323  ###
exception:  327  ###
exception:  330  ###
exception:  340  ###
exception:  356  ###
exception:  373  ###
exception:  378  ###
exception:  383  ###
excep

exception:  3292  ###
exception:  3297  ###
exception:  3301  ###
exception:  3308  ###
exception:  3359  ###
exception:  3362  ###
exception:  3377  ###
exception:  3379  ###
exception:  3420  ###
exception:  3422  ###
exception:  3439  ###
exception:  3445  ###
exception:  3451  ###
exception:  3460  ###
exception:  3463  ###
exception:  3468  ###
exception:  3487  ###
exception:  3495  ###
exception:  3511  ###
exception:  3515  ###
exception:  3517  ###
exception:  3523  ###
exception:  3541  ###
exception:  3560  ###
exception:  3574  ###
exception:  3580  ###
exception:  3602  ###
exception:  3609  ###
exception:  3618  ###
exception:  3621  ###
exception:  3628  ###
exception:  3629  ###
exception:  3640  ###
exception:  3667  ###
exception:  3677  ###
exception:  3695  ###
exception:  3697  ###
exception:  3700  ###
exception:  3701  ###
exception:  3721  ###
exception:  3728  ###
exception:  3761  ###
exception:  3766  ###
exception:  3767  ###
exception:  3771  ###
exception:

exception:  6796  ###
exception:  6811  ###
exception:  6815  ###
exception:  6828  ###
exception:  6837  ###
exception:  6839  ###
exception:  6840  ###
exception:  6853  ###
exception:  6859  ###
exception:  6861  ###
exception:  6864  ###
exception:  6871  ###
exception:  6873  ###
exception:  6877  ###
exception:  6878  ###
exception:  6880  ###
exception:  6881  ###
exception:  6899  ###
exception:  6900  ###
exception:  6929  ###
exception:  6931  ###
exception:  6953  ###
exception:  6954  ###
exception:  6958  ###
exception:  6961  ###
exception:  6973  ###
exception:  6975  ###
exception:  6995  ###
exception:  6997  ###
exception:  6998  ###
exception:  7010  ###
exception:  7026  ###
exception:  7031  ###
exception:  7039  ###
exception:  7055  ###
exception:  7078  ###
exception:  7093  ###
exception:  7104  ###
exception:  7122  ###
exception:  7123  ###
exception:  7125  ###
exception:  7135  ###
exception:  7137  ###
exception:  7166  ###
exception:  7172  ###
exception:

exception:  9770  ###
exception:  9789  ###
exception:  9790  ###
exception:  9808  ###
exception:  9821  ###
exception:  9841  ###
exception:  9842  ###
exception:  9847  ###
exception:  9852  ###
exception:  9868  ###
exception:  9869  ###
exception:  9877  ###
exception:  9880  ###
exception:  9883  ###
exception:  9886  ###
exception:  9912  ###
exception:  9914  ###
exception:  9925  ###
exception:  9928  ###
exception:  9948  ###
exception:  9952  ###
exception:  9959  ###
exception:  9965  ###
exception:  9971  ###
exception:  9973  ###
exception:  9975  ###
exception:  9978  ###
exception:  9990  ###
exception:  9995  ###
exception:  9999  ###
exception:  10012  ###
exception:  10014  ###
exception:  10016  ###
exception:  10018  ###
exception:  10025  ###
exception:  10027  ###
exception:  10042  ###
exception:  10050  ###
exception:  10053  ###
exception:  10055  ###
exception:  10078  ###
exception:  10081  ###
exception:  10089  ###
exception:  10104  ###
exception:  10126 

In [121]:
def extract_quarter(input_soup):
    
    try:
        temp = input_soup.find(id='article_participants')
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*([qQ][1-4])\s*[0-9]{4}.*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)
                
        temp = input_soup.find(id='article_source')        
        for element in temp.findAll('a'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*([qQ][1-4])\s*[0-9]{4}.*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)                
        return None
    except:

        return None
    
def extract_year(input_soup):
    
    try:
        temp = input_soup.find(id='article_participants')
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*[qQ][1-4]\s*([0-9]{4}).*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)
                
        temp = input_soup.find(id='article_source')        
        for element in temp.findAll('a'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('q[1-4]\s*[0-9]{4}',element.contents[0],flags=re.IGNORECASE)):
                    print(element.contents[0])
                    temp_string = re.match(string=element.contents[0],\
                                           pattern='[^qQ]*[qQ][1-4]\s*([0-9]{4}).*')
                    print(temp_string.group(1))
                    call_title = element.contents[0].split()
                    return temp_string.group(1)                
        return None
    except:
        return None  

## Test Some Files

In [ ]:
import csv
fieldnames = ['index','fname', 'status']
# output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/dates')
input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')


with open(output_folder / 'log_extract.csv', 'w') as csvfile:

    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    for ii in range(1,10):
        try:
            total_rows = len(file_list)
            row_start = ii*1
            row_end = (ii*1)+1
            filename = 'exec_out_foringestion_'+str(row_start)+'.csv'

            out_list =[]

            for i in range(row_start,row_end):
                with open(input_folder / file_list[i]) as inf:
#                 with open(collection + '/' + file_list[i]) as inf:
                    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
                        out_list.append(file_list[i])

            out_pd = pd.DataFrame(out_list)
            out_pd.columns=['file']
            out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
            out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
            out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)
            
            out_executives = out_pd                      
            out_executives.drop(['text'],axis=1).to_csv(output_folder / filename,sep=',', encoding = 'utf-8',header=False)
            logwriter.writerow([ii,out_executives.file.head(1).item(),'success'])
        except:
            print('exception: ', ii, ' ###')
            logwriter.writerow([ii,out_executives.file.head(1).item(),'fail'])
            
            pass